<a href="https://colab.research.google.com/github/Prateeklk/Deep-Leaning/blob/main/Chest_XRay.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# importing the libraries
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import to_categorical

In [3]:
train_dir = '/content/drive/MyDrive/health/chest_xray/train'
test_dir = '/content/drive/MyDrive/health/chest_xray/test'

In [4]:
# Define parameters
img_size = (150, 150)
batch_size = 32

In [5]:
# Data augmentation and preprocessing for training
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)


In [6]:
# Data preprocessing for validation and test sets
valid_test_datagen = ImageDataGenerator(rescale=1./255)

In [8]:
# Load the full training data to split into training and validation sets
def load_full_data(directory):
    data_generator = valid_test_datagen.flow_from_directory(
        directory,
        target_size=img_size,
        batch_size=1,
        class_mode='binary',
        shuffle=True
    )
    data = []
    labels = []
    for i in range(len(data_generator.filenames)):
        img, label = data_generator.next()
        data.append(img[0])
        labels.append(label[0])
    return np.array(data), np.array(labels)

full_train_data, full_train_labels = load_full_data(train_dir)


Found 5232 images belonging to 2 classes.


In [9]:
# Split the data into training and validation sets
from sklearn.model_selection import train_test_split
train_data, val_data, train_labels, val_labels = train_test_split(
full_train_data, full_train_labels, test_size=0.2, random_state=42)


In [31]:
len(train_data)

4185

In [33]:
len(train_labels)

4185

In [32]:
len(val_data)

1047

In [34]:
len(val_labels)

1047

In [10]:
# Define generators for training and validation sets
train_generator = train_datagen.flow(train_data, train_labels, batch_size=batch_size)
validation_generator = valid_test_datagen.flow(val_data, val_labels, batch_size=batch_size)


In [11]:
# Load test data
test_generator = valid_test_datagen.flow_from_directory(
    test_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary',
    shuffle=False
)

Found 624 images belonging to 2 classes.


In [42]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.callbacks import ModelCheckpoint, EarlyStopping,ReduceLROnPlateau

# Define the model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.1),
    Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Model checkpoints and early stopping
checkpoint = ModelCheckpoint('best_model.h5', monitor='val_loss', save_best_only=True)
early_stopping = EarlyStopping(monitor='val_loss', patience=10)
#reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.001)



In [43]:
# Train the model
history = model.fit(
    train_generator,
    # steps_per_epoch=len(train_data),# // batch_size,
    steps_per_epoch= 100,
    validation_data=validation_generator,
    validation_steps=25,# // batch_size,
    epochs=40,
    callbacks=[checkpoint, early_stopping] #reduce_lr]
)

Epoch 1/40
100/100 [==============================] - 23s 202ms/step - loss: 0.5887 - accuracy: 0.7403 - val_loss: 0.5811 - val_accuracy: 0.7325
Epoch 2/40
100/100 [==============================] - 20s 199ms/step - loss: 0.5834 - accuracy: 0.7382 - val_loss: 0.6048 - val_accuracy: 0.7337
Epoch 3/40
100/100 [==============================] - 18s 183ms/step - loss: 0.5785 - accuracy: 0.7394 - val_loss: 0.5930 - val_accuracy: 0.7275
Epoch 4/40
100/100 [==============================] - 20s 203ms/step - loss: 0.5736 - accuracy: 0.7419 - val_loss: 0.5813 - val_accuracy: 0.7337
Epoch 5/40
100/100 [==============================] - 18s 184ms/step - loss: 0.5670 - accuracy: 0.7473 - val_loss: 0.5836 - val_accuracy: 0.7300
Epoch 6/40
100/100 [==============================] - 18s 181ms/step - loss: 0.5657 - accuracy: 0.7495 - val_loss: 0.5937 - val_accuracy: 0.7300
Epoch 7/40
100/100 [==============================] - 21s 206ms/step - loss: 0.5721 - accuracy: 0.7435 - val_loss: 0.5759 - val_ac

In [44]:
# Load the best model
from keras.models import load_model

best_model = load_model('best_model.h5')

# Evaluate the model on the test data
loss, accuracy = best_model.evaluate(test_generator)
print(f'Test Accuracy: {accuracy * 100:.2f}%')


20/20 [==============================] - 73s 4s/step - loss: 0.8747 - accuracy: 0.6250
Test Accuracy: 62.50%
